<a href="https://colab.research.google.com/github/dptru10/AI_Materials_Presentation/blob/main/Seminar_1_Optimizing_Models_in_Pytorch_DataParallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.distributed as dist
import subprocess
import os

# Parameters
data_dir = 'chest_xray'  # Directory containing the data
batch_size = 16          # Increased batch size for DataParallel
num_workers = 2
num_epochs = 10
learning_rate = 1e-3

In [4]:
# Check if the file exists
if not os.path.exists('ChestXRay2017.zip'):
    # Download the file using gdown
    subprocess.run(['gdown', '--fuzzy', 'https://drive.google.com/file/d/1jf1XvAeXPD4XAerknz5inxM0StuCNbyX/view?usp=sharing'])
    # Unzip the file
    subprocess.run(['unzip', '-qq', 'ChestXRay2017.zip'])

In [5]:
# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Data loaders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=num_workers) for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

In [6]:
# Model setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 165MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
print("Number of GPUs available:", torch.cuda.device_count())

Number of GPUs available: 1


In [8]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [9]:
import time

def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=10):
    total_start_time = time.time()  # Start timing for the entire training process

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data in the training set
        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Learning rate scheduler step
        scheduler.step()

        # Calculate and print epoch loss and accuracy
        epoch_loss = running_loss / len(dataloaders['train'].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders['train'].dataset)
        print('Train Epoch [{}/{}] - Loss: {:.6f}, Acc: {:.6f}'.format(epoch + 1, num_epochs, epoch_loss, epoch_acc))

    total_training_time = time.time() - total_start_time  # Calculate the total training time
    return total_training_time

In [ ]:
single_gpu_time = train_model(model, dataloaders, criterion, optimizer, scheduler)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Train Epoch [1/10] - Loss: 0.432873, Acc: 0.880925
Train Epoch [2/10] - Loss: 0.183814, Acc: 0.924121
Train Epoch [3/10] - Loss: 0.174132, Acc: 0.931575
Train Epoch [4/10] - Loss: 0.164857, Acc: 0.937691
Train Epoch [5/10] - Loss: 0.144297, Acc: 0.941896
Train Epoch [6/10] - Loss: 0.130264, Acc: 0.953937


# Let's try DataParallel and Compare Timings:

In [ ]:
print(single_gpu_time)

In [ ]:
model = models.resnet18(pretrained=True)
model = nn.DataParallel(model)
model.to(torch.device('cuda'))

In [ ]:
multi_gpu_time = train_model(model, dataloaders, criterion, optimizer, scheduler)
print(multi_gpu_time)

In [ ]:
import matplotlib.pyplot as plt
# Labels for the bars
labels = ['Single GPU', 'Multi GPU']

# Values corresponding to the labels
times = [single_gpu_time, multi_gpu_time]

# Creating the bar plot
plt.figure(figsize=(8, 5))  # You can adjust the figure size as needed
plt.bar(labels, times, color=['blue', 'green'])  # Choose colors as you like

# Adding title and labels
plt.title('GPU Time Comparison')
plt.xlabel('Configuration')
plt.ylabel('Time (seconds)')

# Showing the plot
plt.show()